In [11]:
import numpy as np
import pandas as pd

df = pd.read_csv('data.csv')

df = df.drop(columns = ['Unnamed: 0'])

# According to note 30: "Therefore, to predict returns at month t+1, we use most recent monthly characteristics at the end of month t." <br>
# Hence, **shift return t+1 to serve as response: r(t+1)**.

df['r(t+1)'] = df.groupby('permno')['return'].shift(-1)

### handle missing data

# According to note 30 (bottom of p 2248): "Another issue is missing characteristics, which we replace with the cross-sectional median at each month for each stock, respectively." <br>
# Hence, calculate monthly cross-sectional median for features: **'mom1m', 'mom12m', 'chmom', 'mom36m', 'turn', 'dolvol', 'idiovol', 'beta', 'betasq', 'ep', 'sp', 'agr', 'nincr'**.

df_filled = df.copy()
for feature in ['mom1m', 'mom12m', 'chmom', 'mom36m', 'turn', 'dolvol', 'idiovol', 'beta', 'betasq', 'ep', 'sp', 'agr', 'nincr']:
    df_filled[feature] = df_filled.groupby('Date')[feature].transform(lambda x: x.fillna(x.median()))

df_filled.isna().sum()

df.loc[:, ['mom1m', 'mom12m', 'chmom', 'mom36m', 'turn', 'dolvol', 'idiovol', 'beta', 'betasq', 'ep', 'sp', 'agr', 'nincr']] = df_filled.loc[:,['mom1m', 'mom12m', 'chmom', 'mom36m', 'turn', 'dolvol', 'idiovol', 'beta', 'betasq', 'ep', 'sp', 'agr', 'nincr']]

df['Date'] = pd.to_datetime(df['Date'])

In [12]:
# Set the datetime column as index
df.set_index('Date', inplace=True, drop = True)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

df_scaled = scaler.fit_transform(df)

df_scaled = pd.DataFrame(df_scaled, columns=df.columns)


permno = df['permno'].reset_index(drop = True)

df_scaled['permno'] = permno

df_scaled.index = df.index

In [13]:
df_scaled = df_scaled.reset_index()  # bring back date column

use dataframe: df_scaled<br>
try not to reset the index

In [14]:
companies = pd.read_csv('Companies.csv')
companies.columns

Index(['permno', 'SICCD', 'NCUSIP', 'TICKER', 'COMNAM'], dtype='object')

In [5]:
# print(len(companies['permno'].unique()))

# companies_with_duplicates = companies[companies.duplicated(subset=['permno'], keep=False)]
# companies_with_duplicates

In [19]:
companies.loc[companies['SICCD']=='120']

,permno,SICCD,NCUSIP,TICKER,COMNAM
4554,14955,120,NaN,NaN,UNITED FRUIT CO
5131,15579,120,NaN,NaN,TEXAS INSTRUMENTS INC


In [6]:
companies = companies.sort_values(by='permno')

upd_companies = companies.groupby('permno').tail(1)

# only get the lasted info for each company

In [7]:
# 6669 companies dont have SIC code

# print(upd_companies['SICCD'].isna().values.sum())
# print(len(upd_companies))

In [9]:
merged_data = pd.merge(df_scaled, upd_companies, on='permno', how='left')
merged_data = merged_data.drop(columns = ['NCUSIP', 'TICKER'])

# print(merged_data['SICCD'].isna().values.sum())
# print(merged_data['SICCD'].isna().values.sum()/228)            # drop 135 companies from the dataset that doesn't have sic

merged_data = merged_data[merged_data['SICCD'].notna()]

# merged_data.to_csv('merged_data_clean.csv')